In [134]:
import torch
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T

import torchviz


In [135]:
torch.manual_seed(0)
np.random.seed(0)

In [136]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():   # For Nvidia GPU -- cuda
    device = torch.device('cuda')
elif USE_GPU and torch.backends.mps.is_available(): # For MacOS Ver >=12.3 -- Metal
    device = torch.device("mps")
else:
    device = torch.device('cpu')

print('using device:', device)


using device: cpu


In [137]:
print_every = 100   # Constant to control how frequently we print train loss
N = 2               # the repeating block number of ISML, should be >= 1
D = 75
hidden_size = 100
# emb_length = 200
batch_size = 32

class SecondHalfNet(nn.Module):
    def __init__(self,N,D,hidden_size,device):
        super().__init__()

        self.bilstm_e_list = []
        self.bilstm_c_list = []
        self.fc_e_list = []
        self.fc_c_list = []
        for n in range(N):
            bilstm_e = nn.LSTM(input_size= hidden_size*2+4*n, hidden_size= hidden_size,\
                               num_layers= 1, batch_first=True,bidirectional=True)
            self.bilstm_e_list.append(bilstm_e)

            bilstm_c = nn.LSTM(input_size= hidden_size*2+4*n, hidden_size= hidden_size,\
                               num_layers= 1, batch_first=True,bidirectional=True)
            self.bilstm_c_list.append(bilstm_c)

            fc_e = nn.Linear(hidden_size*2,2)
            # nn.init.kaiming_normal_(fc_e.weight)
            self.fc_e_list.append(fc_e)

            fc_c = nn.Linear(hidden_size*2,2)
            # nn.init.kaiming_normal_(fc_c.weight)
            self.fc_c_list.append(fc_c)


    def forward(self, s1):
        # scores = None
        self.y_e_list = []
        self.y_c_list = []
        s_tmp = s1

        for n in range(N):
            # print(self.bilstm_e_list[n](s_tmp))
            e_lstm_out,_ = self.bilstm_e_list[n](s_tmp)
            y_e = nn.functional.softmax(self.fc_e_list[n](e_lstm_out),dim=2)
            self.y_e_list.append(y_e)

            c_lstm_out,_ = self.bilstm_c_list[n](s_tmp)
            y_c = nn.functional.softmax(self.fc_c_list[n](c_lstm_out),dim=2)
            self.y_c_list.append(y_c)

            s_tmp = torch.cat((s_tmp,y_e,y_c),dim=2)
            
            # print('s_tmp shape',s_tmp.shape)
        
        
        return self.y_e_list,self.y_c_list,s_tmp




In [138]:
def test_SecondHalfNet():
    s1 = torch.rand((batch_size, D, hidden_size*2), dtype=dtype,device=device,\
                    requires_grad=False) 

    model = SecondHalfNet(N,D,hidden_size,device)
    model = model.to(device=device)
    model = torch.compile(model)    # Pytorch 2.0 acceleration
    model.train()
    # model.eval()

    y_e_list,y_c_list,s_final = model(s1)
    # print(len(y_e_list),len(y_c_list),y_e_list[0].shape,y_c_list[0].shape) 

    # torch.onnx.export(model, s1, 'secondhalf.onnx')
    
    # dots = torchviz.make_dot(s_tmp,params=dict(model.named_parameters()),show_attrs=False, show_saved=False)
    # dots.format = 'png'
    # dots.render('secondhalf_modelviz')
    
test_SecondHalfNet()

s_tmp shape torch.Size([32, 75, 204])
s_tmp shape torch.Size([32, 75, 208])
2 2 torch.Size([32, 75, 2]) torch.Size([32, 75, 2])


In [ ]:
def training():
    s1 = torch.rand((batch_size, D, hidden_size*2), dtype=dtype,device=device,\
                    requires_grad=False) 

    model = SecondHalfNet(N,D,hidden_size,device)
    model = model.to(device=device)
    model = torch.compile(model)    # Pytorch 2.0 acceleration
    model.train()
    # model.eval()

    y_e_list,y_c_list,s_final = model(s1)